<a href="https://colab.research.google.com/github/jayliu1016/Xai_3/blob/main/notebooks/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install PyALE statsmodels --upgrade


In [ ]:
import os, pandas as pd
CSV_LOCAL = "data/AmesHousing.csv"
REPO_RAW = "https://raw.githubusercontent.com/jayliu1016/Xai_3/main/data/AmesHousing.csv"

def load_ames():
    if os.path.exists(CSV_LOCAL):
        return pd.read_csv(CSV_LOCAL)
    return pd.read_csv(REPO_RAW)

df = load_ames()
df.shape, df.head()
